# SingleLine Playgroud

This notebook explores some ideas in one-lining Python code while limiting function stack usage.

Warning: lots of hacks ahead!

__Disclaimer__: I realized that this file is a lot less necessary than I initially thought... The target code turned out to be a lot more about using the walrus operator than encoding various stuff with lambda.

## Basic Recursions

Basic recursions in lambda calculus is trivial: you just need a self-reference, usually done with a Y-combinator. However, its counterpart (Z-combinator) in a non-lazy language induces way too many layers of lambda application, which severely reduces the allowed depth of recursion. This project just stupidly uses the walrus operator `:=` to alter the namespace instead of using lambda bindings. This preserves the allowed recursion depth completely.

In [13]:
# source function

def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)


# transformed code

(gcd := (lambda a, b: a if b == 0 else gcd(b, a % b)),)

(<function __main__.<lambda>(a, b)>,)

## Mutual Recursions

Just like the case for basic recursions, the typical lambda encoding of `letrec ... in ...` introduces too many function calls. Instead the walrus operator is abused again...

In [19]:
# source functions

def even(a):
    if a > 0:
        print(a)
        odd(a - 1)
        
def odd(a):
    print(a)
    even(a - 1)
    

# transformed code

(even := lambda a: ((print(a), odd(a - 1)) if a > 0 else (), None)[1], odd := lambda a: (print(a), even(a - 1)))

(<function __main__.<lambda>(a)>, <function __main__.<lambda>(a)>)

## For Loops

For loops introduces complication due to the possibility of an early termination or interruption with `break`, `return` or `continue`. "Pure" for loops, i.e. without any interruption, can be written as a list comprehension:

In [23]:
# source code

acc = 0
for i in range(5):
    acc += i
    

# transformed code

(acc := 0, [acc := acc + i for i in range(5)])

(0, [0, 1, 3, 6, 10])

## Loops with Interruption

For `for` loops with interruption and `while` loops, a right fold is used to handle early termination and `continue`. This is achieved with `filter` on an infinite generator, paired with a preliminary variable to store the accumulated state. Note that since assignment of a variable outside of the scope of a lambda is not possible, the accumulated state is a bundle of all the variables altered in the `while` loop, along with a flag indicating whether the loop should terminate: `[flag, x1, x2, ..., xn]`.

After the loop ends, all variables in the accumulated variable are unpacked into their original names (only for variables that are used afterwards according to the data flow analysis).

While loop:

In [84]:
# source code

x = 0
acc = 0
while x // 500 < 100:
    acc += x
    x += 1

print(acc)


# transformed code

# import can be inlined, but now we're just demonstrating
from itertools import count

(a := [x := 0, acc := 0, not (x // 500 < 100)], next(filter(lambda _: True if a[2] else (a.__setitem__(1, a[1] + a[0]), a.__setitem__(0, a[0] + 1), a.__setitem__(2, not (a[0] // 500 < 100)), False)[-1], count())), acc := a[1], print(acc))

1249975000
1249975000


([50000, 1249975000, True], 50000, 1249975000, None)

`break` and `continue` are implemented by exploiting the short-wiring of the `... if ... else ...` expression.

A control flow graph is created for this purpose during the compilation process.